In [1]:
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in /opt/anaconda3/lib/python3.7/site-packages (0.0)


In [2]:
!pip install joblib

In [3]:
import pandas as pd
from pprint import pprint

In [4]:
df = pd.read_csv("data/exoplanet_data.csv")
df = df.dropna(axis='columns', how='all')
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [5]:
selected_features = df.drop(columns=["koi_disposition",
                            "koi_period_err1", 
                            "koi_period_err2",
                            "koi_time0bk_err1",
                            "koi_time0bk_err2",
                            "koi_steff_err1",
                            "koi_steff_err2",
                            "koi_slogg_err1",
                            "koi_slogg_err2",
                            "koi_srad_err1",
                            "koi_srad_err2"],
                            axis=1)

In [6]:
y = df["koi_disposition"]
X = selected_features

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_time0bk,koi_impact,koi_impact_err1,koi_impact_err2,koi_duration,...,koi_insol_err1,koi_insol_err2,koi_model_snr,koi_tce_plnt_num,koi_steff,koi_slogg,koi_srad,ra,dec,koi_kepmag
6122,0,0,0,0,6.768901,133.077240,0.150,0.305,-0.150,3.61600,...,204.89,-103.87,10.8,1,5737,4.327,1.125,294.40472,39.351681,14.725
6370,0,1,0,1,0.733726,132.020050,0.291,0.193,-0.291,2.30900,...,2253.61,-677.78,13.8,1,5855,4.578,0.797,284.50391,42.463860,15.770
2879,1,0,0,0,7.652707,134.460380,0.970,0.879,-0.942,79.89690,...,195.16,-64.34,254.3,1,6328,4.481,0.963,295.50211,38.983540,13.099
107,0,0,0,0,7.953547,174.662240,0.300,0.145,-0.300,2.63120,...,7.15,-10.12,38.4,1,4768,4.536,0.779,291.15878,40.750271,15.660
29,0,0,0,0,4.959319,172.258529,0.831,0.016,-0.004,2.22739,...,146.52,-93.21,696.5,1,5712,4.359,1.082,292.16705,48.727589,15.263


In [9]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [10]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder().fit(y_train) 

y_train_encoded = encoder.transform(y_train)
y_test_encoded = encoder.transform(y_test)
y_test_encoded

array([2, 0, 2, ..., 1, 1, 1])

In [11]:
disposition=pd.DataFrame({"Actual": y_test, "Encoded": y_test_encoded})
disposition.head()

,Actual,Encoded
4982,FALSE POSITIVE,2
4866,CANDIDATE,0
2934,FALSE POSITIVE,2
5007,FALSE POSITIVE,2
3869,FALSE POSITIVE,2


In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=200)
model = model.fit(X_train_scaled, y_train_encoded)

In [13]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test_encoded)}")

Training Data Score: 1.0
Testing Data Score: 0.9010297482837528


In [14]:
pprint(model.get_params())

{'bootstrap': True,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [15]:
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion': ['entropy'],
              'max_depth': [74, 75, 76], 
              'min_weight_fraction_leaf': [0],
              'min_samples_split': [8]}


grid = GridSearchCV(model, param_grid, verbose=1)

In [16]:
grid.fit(X_train_scaled, y_train_encoded)

Fitting 3 folds for each of 3 candidates, totalling 9 fits


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:   22.6s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=200, n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             i

In [17]:
print(grid.best_params_)
print(grid.best_score_)

{'criterion': 'entropy', 'max_depth': 74, 'min_samples_split': 8, 'min_weight_fraction_leaf': 0}
0.897196261682243


In [18]:
model_best_param = RandomForestClassifier(criterion = grid.best_params_['criterion'],
                                          max_depth = grid.best_params_['max_depth'],
                                          min_weight_fraction_leaf = grid.best_params_['min_weight_fraction_leaf'],
                                          min_samples_split = grid.best_params_['min_samples_split'])

model_best_param.fit(X_train_scaled, 
                     y_train_encoded)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
                       max_depth=74, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=8,
                       min_weight_fraction_leaf=0, n_estimators=10, n_jobs=None,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [19]:
from sklearn.metrics import classification_report

predictions = model_best_param.predict(X_test_scaled)
print(classification_report(y_test_encoded, predictions,
                            target_names=[ 'Candidate', 'Confirmed', 'False Positive']))
print(f"Testing Data Score: {model_best_param.score(X_test_scaled, y_test_encoded)}")

                precision    recall  f1-score   support

     Candidate       0.80      0.77      0.78       411
     Confirmed       0.83      0.83      0.83       484
False Positive       0.97      0.99      0.98       853

      accuracy                           0.89      1748
     macro avg       0.87      0.86      0.86      1748
  weighted avg       0.89      0.89      0.89      1748

Testing Data Score: 0.8918764302059496


In [20]:
predictions = model_best_param.predict(X_test_scaled)

predictions_df=pd.DataFrame({"Actual": y_test_encoded, "Prediction": predictions, })

results=predictions_df.apply(pd.Series.value_counts)
disposition = ['False Positive', 'Confirmed', 'Candidate',] 
results['Disposition']=disposition
results

,Actual,Prediction,Disposition
2,853,870,False Positive
1,484,483,Confirmed
0,411,395,Candidate


In [21]:
features = X.columns
sorted(zip(model.feature_importances_, features), reverse=True)

[(0.11515755895851527, 'koi_fpflag_co'),
 (0.10627908641388274, 'koi_fpflag_nt'),
 (0.07374078346286402, 'koi_model_snr'),
 (0.06799525424478971, 'koi_fpflag_ss'),
 (0.053392090614290795, 'koi_prad'),
 (0.049306762305598835, 'koi_duration_err2'),
 (0.04823644042691509, 'koi_duration_err1'),
 (0.0425846223844516, 'koi_prad_err1'),
 (0.04031406111679286, 'koi_fpflag_ec'),
 (0.036053220188236465, 'koi_prad_err2'),
 (0.030937610678555334, 'koi_period'),
 (0.02799429958667517, 'koi_depth'),
 (0.025621415426610828, 'koi_insol_err1'),
 (0.025293183431087416, 'koi_duration'),
 (0.023644286800804045, 'koi_impact'),
 (0.022850422557436866, 'koi_teq'),
 (0.020735998327374612, 'koi_depth_err2'),
 (0.018888874729719018, 'koi_time0bk'),
 (0.01834567565547471, 'koi_insol'),
 (0.017850859920516327, 'koi_depth_err1'),
 (0.01736913581694522, 'koi_insol_err2'),
 (0.015404612644791196, 'ra'),
 (0.014784979508300536, 'dec'),
 (0.014603685519750265, 'koi_kepmag'),
 (0.014538846165688098, 'koi_steff'),
 (0.0

In [22]:
import joblib
filename = 'models/random_forest.sav'
joblib.dump(model, filename)

['models/random_forest.sav']